In [1]:
include("./src/parse_file.jl")

Completed analyis of 63 reads in chr1
Completed analyis of 37 reads in chr2


Dict{String, SparseMatrixCSC{Int64, Int64}} with 2 entries:
  "chr2" => sparse([2, 2, 4, 6, 2, 4, 6, 4, 6, 4  …  15, 15, 16, 16, 17, 18, 17…
  "chr1" => sparse([2, 2, 2, 2, 2, 3, 3, 1, 1, 4  …  23, 24, 25, 26, 27, 21, 22…

In [2]:
path = "./testing/test_epiread_small.bed"
big_bgz_path = "/Volumes/projects/laird/nathan/projects/julia/epiread/testing/test_epiread.bed.bgz"
part_path = "/Volumes/projects/laird/nathan/projects/julia/epiread/testing/part_epiread.bed"


"/Volumes/projects/laird/nathan/projects/julia/epiread/testing/part_epiread.bed"

In [3]:
results = analyze_file(path)

Completed analyis of 63 reads in chr1
Completed analyis of 37 reads in chr2


Dict{String, SparseMatrixCSC{Int64, Int64}} with 2 entries:
  "chr2" => sparse([2, 2, 4, 6, 2, 4, 6, 4, 6, 4  …  15, 15, 16, 16, 17, 18, 17…
  "chr1" => sparse([2, 2, 2, 2, 2, 3, 3, 1, 1, 4  …  23, 24, 25, 26, 27, 21, 22…

In [4]:
x= results["chr1"]

27×631931 SparseMatrixCSC{Int64, Int64} with 105 stored entries:
⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠈⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠒⠂⠐

In [5]:
x[4,:]

631931-element SparseVector{Int64, Int64} with 5 stored entries:
  [136571]  =  77
  [136583]  =  77
  [136618]  =  77
  [136620]  =  77
  [136632]  =  77

In [6]:
function get_unsparse_nonzero_rows(matrix::SparseMatrixCSC{Int64, Int64})
	nzrows = unique(matrix.rowval)
	if !isempty(nzrows)
		return Matrix(matrix)[nzrows,:]
	else 
		return Matrix{Int64}(undef, 0, 0)
	end
end

get_unsparse_nonzero_rows (generic function with 1 method)

In [18]:
window_size = 4

for i in 1:window_size:200000
	try
		a = get_unsparse_nonzero_rows(x[:,i:i+window_size-1])
		if !isempty(a)
			# count_epialleles(x)
			println(i)
			@show a
		end
	catch
	end
end

136289
a = [0 0 77 0]
136309
a = [0 0 0 77]
136325
a = [0 77 0 77]
136337
a = [0 0 0 77]
136485
a = [77 0 0 0]
136505
a = [0 77 0 0]
136517
a = [0 0 0 77]
136533
a = [0 77 0 0]
136569
a = [0 0 77 0]
136581
a = [0 0 77 0; 0 0 77 0; 0 0 77 0]
136617
a = [0 77 0 77; 0 77 0 77; 0 77 0 77; 0 0 0 85]
136629
a = [0 0 0 77; 0 0 0 77; 0 0 0 77; 0 0 0 77]
136713
a = [0 0 0 85]
136717
a = [0 85 0 0]
136777
a = [0 77 0 0; 0 77 0 0]
136797
a = [85 0 0 0; 85 0 0 0]
180609
a = [0 0 0 77]
180637
a = [0 85 0 0]
180649
a = [0 77 0 0]
180673
a = [0 77 0 0; 0 77 0 0]
190909
a = [0 85 0 0]
191437
a = [0 77 0 0; 0 77 0 0]
191489
a = [0 0 77 0; 0 0 77 0]


In [16]:
collect(1:4:20)

5-element Vector{Int64}:
  1
  5
  9
 13
 17

In [26]:
a = [0 77 0 77; 0 77 0 77; 0 65 0 77]

3×4 Matrix{Int64}:
 0  77  0  77
 0  77  0  77
 0  65  0  77

In [41]:
function count_alleles(matrix::Matrix{Int64}, window_size::Int)
	diff = zeros(Int, window_size)
	for col in 1:window_size
		diff[col] = length(unique(matrix[:,col]))
	end
	return maximum(diff)
end

count_alleles (generic function with 2 methods)

In [42]:
count_alleles(a, 4)

2